In [1]:
!git clone https://github.com/tensorflow/tcav.git tcav
%cd tcav
!ls

 100% |████|  762.4Mb/762.4Mb [5.9s elapsed, 0s remaining, 133.6Mb/s]      
Extracting dataset...
Parsing dataset metadata
Found 1000 samples
Dataset info written to 'C:\Users\vpand\fiftyone\imagenet-sample\info.json'
Loading 'imagenet-sample'
 100% |███████████████| 1000/1000 [650.7ms elapsed, 0s remaining, 1.5K samples/s]      
Dataset 'imagenet-sample' created


In [17]:
# %pwd
%cd tcav_examples/image_models/imagenet 
%run download_and_make_datasets.py --source_dir=YOUR_FOLDER --number_of_images_per_folder=10 --number_of_random_folders=10

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [3]:
import tcav.activation_generator as act_gen
import tcav.cav as cav
import tcav.model  as model
import tcav.tcav as tcav
import tcav.utils as utils
import tcav.utils_plot as utils_plot # utils_plot requires matplotlib
import os 
import tensorflow as tf

tf.Tensor(
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]], shape=(10000, 2), dtype=float32)


In [27]:
# This is the name of your model wrapper (InceptionV3 and GoogleNet are provided in model.py)
model_to_run = 'GoogleNet'
# the name of the parent directory that results are stored (only if you want to cache)
project_name = 'tcav_class_test'
working_dir = '/content/tcav/tcav'
# where activations are stored (only if your act_gen_wrapper does so)
activation_dir =  working_dir+ '/activations/'
# where CAVs are stored. 
# You can say None if you don't wish to store any.
cav_dir = working_dir + '/cavs/'
# where the images live. 
source_dir = '/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER'
bottlenecks = [ 'mixed4c']  # @param 
      
utils.make_dir_if_not_exists(activation_dir)
utils.make_dir_if_not_exists(working_dir)
utils.make_dir_if_not_exists(cav_dir)

# this is a regularizer penalty parameter for linear classifier to get CAVs. 
alphas = [0.1]   

target = 'zebra'  
concepts = ["dotted","striped","zigzagged"]   

(0,)
[]


In [28]:
unpickled_file = unpickle(r"C:\Users\vpand\Documents\College\Fall 2021\CS1470 HW\hw2-cnn-vpandiarajan20\data\train")
inputs = unpickled_file[b'data']
labels = unpickled_file[b'labels']
labels_coords = np.nonzero((labels == 3) | (labels == 5))
labels = labels[labels_coords];
inputs = inputs[labels_coords] / 255;
inputs = tf.reshape(inputs, (-1, 3, 32 ,32))
inputs = tf.transpose(inputs, perm=[0,2,3,1])
labels = np.where(labels == 3, 0, 1)
o_hot_labels = tf.one_hot(labels, 2)


In [57]:
inputs = inputs[0:64]
print(tf.shape(inputs))
filters_1 = tf.Variable(tf.random.truncated_normal([5,5,3,16], stddev=.1, dtype=tf.double))
conv_1 = tf.nn.conv2d(inputs, filters_1, strides=[1, 1, 1, 1], padding='VALID')
print(tf.shape(conv_1))
mean_1, variance_1 = tf.nn.moments(conv_1, [0,1,2])
batch_normalize_1 = tf.nn.batch_normalization(conv_1, mean_1, variance_1, None, None, variance_epsilon=1e-5)
post_relu_1 = tf.nn.relu(batch_normalize_1)
post_max_pooling_1 = tf.nn.max_pool(post_relu_1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')
print(tf.shape(post_max_pooling_1))

filters_2 = tf.Variable(tf.random.truncated_normal([5,5,16,20], stddev=.1, dtype=tf.double))        
conv_2 = tf.nn.conv2d(post_max_pooling_1, filters_2, strides=[1, 2, 2, 1], padding='SAME')
print(tf.shape(conv_2))
mean_2, variance_2 = tf.nn.moments(conv_2, [0,1,2])
batch_normalize_2 = tf.nn.batch_normalization(conv_2, mean_2, variance_2, None, None, variance_epsilon=1e-5)
post_relu_2 = tf.nn.relu(batch_normalize_2)
post_max_pooling_2 = tf.nn.max_pool(post_relu_2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
print(tf.shape(post_max_pooling_2))

filters_3 = tf.Variable(tf.random.truncated_normal([3,3,20,20], stddev=.1, dtype=tf.double))
conv_3 = tf.nn.conv2d(post_max_pooling_2, filters_3, strides=[1,1,1,1], padding="SAME")
print(tf.shape(conv_3))
mean_3, variance_3 = tf.nn.moments(conv_3, [0,1,2])
batch_normalize_3 = tf.nn.batch_normalization(conv_3, mean_3, variance_3, None, None, variance_epsilon=1e-5)
post_relu_3 = tf.nn.relu(batch_normalize_3)
print(tf.shape(post_relu_3))



tf.Tensor([64 32 32  3], shape=(4,), dtype=int32)
tf.Tensor([64 28 28 16], shape=(4,), dtype=int32)
tf.Tensor([64 14 14 16], shape=(4,), dtype=int32)
tf.Tensor([64  7  7 20], shape=(4,), dtype=int32)
tf.Tensor([64  4  4 20], shape=(4,), dtype=int32)
tf.Tensor([64  4  4 20], shape=(4,), dtype=int32)
tf.Tensor([64  4  4 20], shape=(4,), dtype=int32)


In [38]:
rank_3_tensor = tf.constant([
  [[0, 1, 2, 3, 4],
   [5, 6, 7, 8, 9]],
  [[10, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
  [[20, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]],])

print(tf.tensordot(rank_3_tensor, rank_3_tensor, axes = 3))

tf.Tensor(8555, shape=(), dtype=int32)


In [55]:
inputImage = tf.ones([64, 5, 5, 3])
inputFilters = tf.ones([5, 5, 3, 16])
# print(tf.tensordot(inputImage, inputFilters, ([1,2,3], [0, 1, 2])))
outputs = np.zeros((64, 32, 32, 16))
tensorOut = tf.tensordot(inputImage, inputFilters, ([1,2,3], [0, 1, 2]))
outputs[:, 0, 0, :] = tensorOut
print(outputs[0, :, :, :])

[[[75. 75. 75. ... 75. 75. 75.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]

 [[ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]

 [[ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]

 ...

 [[ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]

 [[ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  

In [75]:
import math
def conv2d(inputs, filters, strides, padding):
	"""
	Performs 2D convolution given 4D inputs and filter Tensors.
	:param inputs: tensor with shape [num_examples, in_height, in_width, in_channels]
	:param filters: tensor with shape [filter_height, filter_width, in_channels, out_channels]
	:param strides: MUST BE [1, 1, 1, 1] - list of strides, with each stride corresponding to each dimension in input
	:param padding: either "SAME" or "VALID", capitalization matters
	:return: outputs, NumPy array or Tensor with shape [num_examples, output_height, output_width, output_channels]
	"""
	inputs_shapes = tf.shape(inputs)
	inputs_shapes = np.array(inputs_shapes)
	print(inputs_shapes)

	num_examples = inputs_shapes[0]
	in_height = inputs_shapes[1]
	in_width = inputs_shapes[2]
	input_in_channels = inputs_shapes[3]
	
	filter_shapes = tf.shape(filters)
	filter_shapes = np.array(filter_shapes)
	print(np.array(filter_shapes))
	

	filter_height = filter_shapes[0]
	filter_width = filter_shapes[1]
	filter_in_channels = filter_shapes[2]
	filter_out_channels = filter_shapes[3]

	print(filter_out_channels)

	strides = np.array(strides)

	num_examples_stride = strides[0]
	strideY = strides[1]
	strideX = strides[2]
	channels_stride = strides[3]

	assert input_in_channels == filter_in_channels, "input in channels are not equal to filter in channels"

	# Cleaning padding input
	if padding == "SAME":
		padX = math.floor((filter_height-1)/2)
		padY = math.floor((filter_height-1)/2)
	else:
		padX = 0
		padY = 0
	# Calculate output dimensions
	output_height = math.floor(((in_height + 2*padY - filter_height) / strideY + 1))
	output_width = math.floor(((in_width + 2*padX - filter_width) / strideX + 1))
	print(type(output_height))
	outputs = np.zeros((num_examples, output_width, output_height, filter_out_channels))

	inputs = np.pad(inputs, ((0, 0), (padX, padX), (padY, padY), (0, 0)), 'constant')

	for i in range(output_height):
		for j in range(output_width):
			outputs[:, i, j, :] = tf.tensordot(inputs[:, i:i+5, i:i+5, :], filters, ([1,2,3], [0, 1, 2]))
	
	return outputs

filters_1 = tf.Variable(tf.random.truncated_normal([5,5,3,16], stddev=.1, dtype=tf.double))
my_conv = conv2d(inputs, filters_1, [1, 1, 1, 1], padding = "SAME")
conv = tf.nn.conv2d(inputs, filters_1, strides=[1, 1, 1, 1], padding="SAME")
print(np.sum(conv - my_conv))

[64 32 32  3]
[ 5  5  3 16]
16
<class 'int'>
2241.5220263007527
